# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста. Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).

Подсказки:

- нужно будет добавить еще один тэг <start>  
- еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
- тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from collections import Counter
from nltk.tokenize import sent_tokenize
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

In [2]:
news = open('lenta.txt', encoding='utf-8').read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [4]:
norm_news = normalize(news)

In [5]:
vocab_news = Counter(norm_news)

In [6]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [7]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)][:-100]
sentences_test = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)][-100:]

In [8]:
unigrams_news = Counter()
bigrams_news = Counter()
threegrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    threegrams_news.update(ngrammer(sentence, n=3))

In [9]:
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))

id2word_unigrams_news = list(unigrams_news)
word2id_unigrams_news = {word:i for i, word in enumerate(id2word_unigrams_news)}

id2word_bigrams_news = list(bigrams_news)
word2id_bigrams_news = {word:i for i, word in enumerate(id2word_bigrams_news)}

for ngram in threegrams_news:
    wordl, word2, word3 = ngram.split() 
    bigram = wordl + ' ' + word2
    matrix_news[word2id_bigrams_news[bigram], word2id_unigrams_news[word3]] = (threegrams_news[ngram]/bigrams_news[bigram])

In [10]:
matrix_news = csr_matrix(matrix_news)

In [11]:
def generate(matrix, id2word_bigrams_news, id2word_unigrams_news, word2id_bigrams_news, n=200, start='<start> <start>'):
    text = []
    current_idx = word2id_bigrams_news[start]
    for i in range(n):
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])
        text.append(id2word_unigrams_news[chosen])
        if id2word_unigrams_news[chosen] == '<end>':
            current_idx = word2id_bigrams_news[start]
        else:
            part = id2word_bigrams_news[current_idx] + ' ' + id2word_unigrams_news[chosen]
            part = ' '.join(part.split()[1:])
            current_idx = word2id_bigrams_news[part]
    
    return ' '.join(text)

In [22]:
generated_text = generate(matrix_news, id2word_bigrams_news, id2word_unigrams_news, word2id_bigrams_news).replace('<end>', '\n\n')

In [23]:
print(generated_text)

не совсем понимаю зачем распространяется подобная информация 

 замгенпрокурора россии василий средин это стало неожиданностью для американских синоптиков 

 как никто из моряков не могли найти англичане при обыске было обнаружено захоронение принадлежит лишенному сана католическому священнику доменику катарибао одному из членов российской делегации глава цик отметил что такая ситуация затрудняет взаимодействие сторон 

 оформлением храма занимались творческие коллективы 

 произошел перегрев подшипников и пошел дым 

 они обсудили свое дальнейшее сотрудничество в расследовании дел связанных с реализацией положений закона о налоге на доходы физических лиц 

 четыре члена экипажа и пассажиров выбросило наружу 

 в других развитых странах отметил вешняков 

 к первым безусловно можно отнести слабые структурную и налоговую политику 

 напомним что один из двух партий вольфгангом шюсселем и йоргом хайдером подчеркивается что современное законодательство обязывает полицию безопасности обнар

In [14]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    return p ** (-1/N) 

In [15]:
_perplexity = []
for sent in sentences_test:
    probs = []
    for ngram in ngrammer(sent, 3):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' '+word2
        
        if ngram in threegrams_news and bigram in bigrams_news:
            probs.append(np.log(threegrams_news[ngram]/bigrams_news[bigram]))
        else:
            probs.append(np.log(0.00001))
    if perplexity(probs)!= np.inf: 
        _perplexity.append(perplexity(probs))

In [16]:
np.mean(_perplexity)

20089.202373196247

## Задание № 2* (2 балла).

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

__Ответ: Можно создать фиксироанный словарь, а все слова, которые не входят в него, пометить тэгом "UNK" на этапе нормализации. Затем, для этих слов рассчитать вероятность как для любых других. Либо можно неявно создать словарь, а малочастнотные слова заменить на "UNK".__

2. Что такое сглаживание (smoothing)?

__Ответ: Сглаживание - это перераспределение вероятностей, то есть понижение вероятности для частых случаев в пользу случаев, которые мы раньше не видели или в которых не уверены (неизвестный контекст).__